## Code to generate manifest.csv
- Based on this [schema](https://www.neurobagel.org/documentation/mr_proc/configs/)

### QPN naming convention
- `visit`: timepoint of **ANY** clinical (i.e. UPDRS, MoCA, Neuropsy) data collected 
    - e.g. V01, V02 etc. 
- `session`: timepoint of MRI collection
    - e.g. ses-01, ses-02 etc.
- `event`: timepoint relative to a consensus baseline (used for inter-modality i.e. MRI vs clinical data harmonization) 
    - e.g. baseline, m06, m12 etc. 

In [12]:
import pandas as pd
import numpy as np

### Paths

In [13]:
releases_dir = "/home/nikhil/projects/Parkinsons/qpn/releases/"

previous_release = "Jan_2024"
current_release = "March_2024" # No new data yet (20 March 2024)

# Previous mr_proc manifest
previous_manifest_csv = f"{releases_dir}{previous_release}/tabular/manifest.csv"

# Current mr_proc manifest
current_manifest_csv = f"{releases_dir}{current_release}/tabular/manifest.csv"

# Current recruit manifest
current_recruit_manifest_xls = f"{releases_dir}{current_release}/tabular/demographics/Suivi_RPQ.xlsx"

### Read recruitment manifest from previous release

In [14]:
previous_recruit_manifest_df = pd.read_csv(previous_manifest_csv)
previous_recruit_manifest_df["participant_id"] = previous_recruit_manifest_df["participant_id"].str.strip()
nipoppy_participants_previous = previous_recruit_manifest_df["participant_id"].dropna().unique()
n_nipoppy_participants_previous = len(nipoppy_participants_previous)

print(f"number of participants from previous nipoppy release: {n_nipoppy_participants_previous}")
previous_recruit_manifest_df.head()

number of participants from previous nipoppy release: 298


,participant_id,visit,session,datatype
0,PD00016,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
1,PD00020,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
2,PD00032,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
3,PD00048,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
4,PD00119,MRI_v1,ses-01,"['anat','dwi','fmap','func']"


### Read latest recruitment manifest

In [15]:
suivi_df

,participant_id,IRM01_date,IRM01_PD,IRM01_CTRL,IRM01_RBD,IRM01_OTHER,IRM02_date,IRM02_PD,IRM02_CTRL,IRM02_RBD,IRM02_OTHER,IRM03_date,IRM03_PD,IRM03_CTRL,IRM03_RBD,IRM03_OTHER,visit_interval (V2-V1) in days,visit_interval (V3-V2) in days
15,PD00016,2019-07-26,1.0,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
19,PD00020,2018-05-12,1.0,0.0,0.0,NaN,2022-11-24,1,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,1657.0,NaN
31,PD00032,2019-07-24,1.0,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
47,PD00048,2019-08-21,1.0,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
117,PD00119,2018-08-13,1.0,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,MNI0565,2023-09-11,0.0,1.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2231,MNI0602,2023-11-22,1.0,0.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2234,MNI0605,2023-12-15,0.0,1.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2236,MNI0607,NaT,1.0,0.0,0.0,0.0,NaT,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
col_range = "A:R"

col_rename_dict = {
    "subj_id":"participant_id",
    "IRM01\n(J-M-A)":"IRM01_date", "#IRM 1\n PD":"IRM01_PD", "#IRM 1\n CTRL":"IRM01_CTRL", 
    "# IRM 1\n RBD":"IRM01_RBD", "# IRM 1\nOTHER":"IRM01_OTHER",
    "IRM 2 \n(J-M-A)":"IRM02_date", "#IRM 2\n PD":"IRM02_PD", "#IRM 2\n CTRL":"IRM02_CTRL", 
    "# IRM 2\n RBD":"IRM02_RBD", "# IRM 2 OTHER":"IRM02_OTHER",
    "IRM 3\n(J-M-A)":"IRM03_date", "#IRM 3\n PD":"IRM03_PD", "#IRM 3\n CTRL":"IRM03_CTRL", 
    "# IRM 3\n RBD":"IRM03_RBD", "# IRM 3 OTHER":"IRM03_OTHER"
    }

useful_cols = col_rename_dict.values()

suivi_df = pd.read_excel(current_recruit_manifest_xls,sheet_name="En cours", engine='openpyxl', usecols=col_range)
suivi_df = suivi_df.rename(columns=col_rename_dict)[useful_cols].copy()

# remove the row with tally
suivi_df = suivi_df.drop([0])

# remove rows without participant_id
suivi_df = suivi_df.dropna(axis=0, subset=["participant_id"])
suivi_df = suivi_df[~suivi_df["participant_id"].astype(str).isin(["0"])] 

# remove subjects without imaging data
suivi_df = suivi_df[(suivi_df["IRM01_PD"] == 1) | (suivi_df["IRM01_CTRL"] == 1) | 
                    (suivi_df["IRM01_RBD"] == 1) | (suivi_df["IRM01_OTHER"] == 1) |
                    (suivi_df["IRM02_PD"] == 1) | (suivi_df["IRM02_CTRL"] == 1) | 
                    (suivi_df["IRM02_RBD"] == 1) |(suivi_df["IRM02_OTHER"] == 1) |
                    (suivi_df["IRM03_PD"] == 1) | (suivi_df["IRM03_CTRL"] == 1) | 
                    (suivi_df["IRM03_RBD"] == 1) |(suivi_df["IRM03_OTHER"] == 1) ]


# fix participant_id formatting issues
# Some rows have Dx in participant_id and one participant with two IDs with "="
possible_delimiters = [" ", "(", "=", "\n"]
for delim in possible_delimiters:        
    suivi_df["participant_id"] = suivi_df["participant_id"].str.strip().str.split(pat=delim, n=1, expand=True)[0]

# nipoppy_participants_current
nipoppy_participants_current = suivi_df["participant_id"].dropna().unique()

suivi_df

,participant_id,IRM01_date,IRM01_PD,IRM01_CTRL,IRM01_RBD,IRM01_OTHER,IRM02_date,IRM02_PD,IRM02_CTRL,IRM02_RBD,IRM02_OTHER,IRM03_date,IRM03_PD,IRM03_CTRL,IRM03_RBD,IRM03_OTHER
15,PD00016,26-07-2019,1.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
19,PD00020,2018-05-12 00:00:00,1.0,0.0,0.0,NaN,24-11-2022,1,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN
31,PD00032,24-07-2019,1.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
47,PD00048,21-08-2019,1.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
117,PD00119,13-08-2018,1.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,MNI0565,2023-09-11 00:00:00,0.0,1.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2231,MNI0602,2023-11-22 00:00:00,1.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2234,MNI0605,15-12-2023,0.0,1.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
2236,MNI0607,00:00:00,1.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


In [17]:
suivi_df[suivi_df["participant_id"] == "MNI0342"]

,participant_id,IRM01_date,IRM01_PD,IRM01_CTRL,IRM01_RBD,IRM01_OTHER,IRM02_date,IRM02_PD,IRM02_CTRL,IRM02_RBD,IRM02_OTHER,IRM03_date,IRM03_PD,IRM03_CTRL,IRM03_RBD,IRM03_OTHER
1890,MNI0342,25-11-2022,0.0,0.0,0.0,1.0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


### Set date columns and check visit order

In [18]:
# set date columns to datetime
# has mixed types, but auto formatting + coerce works fine here

# explicitely set 0 to nan to avoid origin issues (date: 0 is 1970-01-01)
suivi_df["IRM01_date"] = suivi_df["IRM01_date"].replace(0, np.nan)
suivi_df["IRM02_date"] = suivi_df["IRM02_date"].replace(0, np.nan)
suivi_df["IRM03_date"] = suivi_df["IRM03_date"].replace(0, np.nan)

suivi_df["IRM01_date"] = pd.to_datetime(suivi_df["IRM01_date"], errors="coerce")
suivi_df["IRM02_date"] = pd.to_datetime(suivi_df["IRM02_date"], errors="coerce")
suivi_df["IRM03_date"] = pd.to_datetime(suivi_df["IRM03_date"], errors="coerce")

# Check visit orders
suivi_df["visit_interval (V2-V1) in days"] = suivi_df["IRM02_date"] - suivi_df["IRM01_date"]
suivi_df["visit_interval (V2-V1) in days"] = suivi_df["visit_interval (V2-V1) in days"].dt.days
suivi_df["visit_interval (V3-V2) in days"] = suivi_df["IRM03_date"] - suivi_df["IRM02_date"]
suivi_df["visit_interval (V3-V2) in days"] = suivi_df["visit_interval (V3-V2) in days"].dt.days


visits_wit_wrong_order_df = suivi_df[(suivi_df["visit_interval (V2-V1) in days"] < 0) | 
                                     (suivi_df["visit_interval (V3-V2) in days"] < 0)]


print(f"Participants with wrong visit order: ({len(visits_wit_wrong_order_df)}) :{visits_wit_wrong_order_df['participant_id'].values}")


Participants with wrong visit order: (1) :['PD00849']


/tmp/ipykernel_28865/525785016.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  suivi_df["IRM01_date"] = pd.to_datetime(suivi_df["IRM01_date"], errors="coerce")
/tmp/ipykernel_28865/525785016.py:10: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  suivi_df["IRM02_date"] = pd.to_datetime(suivi_df["IRM02_date"], errors="coerce")


## PD00849 has wrong visit order

In [22]:
suivi_df[suivi_df["participant_id"] == "PD00849"]

,participant_id,IRM01_date,IRM01_PD,IRM01_CTRL,IRM01_RBD,IRM01_OTHER,IRM02_date,IRM02_PD,IRM02_CTRL,IRM02_RBD,IRM02_OTHER,IRM03_date,IRM03_PD,IRM03_CTRL,IRM03_RBD,IRM03_OTHER,visit_interval (V2-V1) in days,visit_interval (V3-V2) in days
844,PD00849,2019-05-27,1.0,0.0,0.0,NaN,2022-11-23,1,0.0,0.0,NaN,1970-01-01 00:00:00.000045218,1.0,0.0,0.0,NaN,1276.0,-19319.0


### Get new participants per session
This is based on a valid date for MRI visit

In [23]:
total_partcipants_additions = list(set(nipoppy_participants_current) - set(nipoppy_participants_previous))
print(f"number of new participants: {len(total_partcipants_additions)}")

MRI_v1_participants = suivi_df[(suivi_df["IRM01_PD"] == 1) | (suivi_df["IRM01_CTRL"] == 1) | 
                               (suivi_df["IRM01_RBD"] == 1) | (suivi_df["IRM01_OTHER"] == 1)]["participant_id"].dropna().unique()
MRI_v2_participants = suivi_df[(suivi_df["IRM02_PD"] == 1) | (suivi_df["IRM02_CTRL"] == 1) | 
                               (suivi_df["IRM02_RBD"] == 1) | (suivi_df["IRM02_OTHER"] == 1)]["participant_id"].dropna().unique()
MRI_v3_participants = suivi_df[(suivi_df["IRM03_PD"] == 1) | (suivi_df["IRM03_CTRL"] == 1) |
                                (suivi_df["IRM03_RBD"] == 1) | (suivi_df["IRM03_OTHER"] == 1)]["participant_id"].dropna().unique()


visit_participant_dict = {"MRI_v1": MRI_v1_participants, "MRI_v2": MRI_v2_participants, "MRI_v3": MRI_v3_participants}

print(f"MRI_v1_participants: {len(MRI_v1_participants)}, MRI_v2_participants: {len(MRI_v2_participants)}, MRI_v3_participants: {len(MRI_v3_participants)}")


number of new participants: 5
MRI_v1_participants: 303, MRI_v2_participants: 52, MRI_v3_participants: 4


### Generate current_manifest_df

- Populate manifest with available clinical visits / MRI sessions
- Datatypes: Assuming QPN has all 4 BIDS datatypes: ["anat","dwi","fmap","func"]

- Sample `manifest.csv`

| participant_id | participant_dicom_dir | visit | session | datatype                     | bids_id |
|----------------|-----------------------|-------|---------|------------------------------|---------|
| 001            | MyStudy_001_V2021      | V01   | ses-01  | ["anat","dwi","fmap","func"] | sub-001 |
| 001            | MyStudy_001_V2022      | V02   | ses-02  | ["anat"]                     | sub-001 |
| 002            | MyStudy_002_V2021      | V01   | ses-01  | ["anat","dwi"]               | sub-002 |
| 002            | MyStudy_002_V2024      | V03   | ses-03  | ["anat","dwi"]               | sub-002 |

In [24]:
visit_labels = visit_participant_dict.keys()
visit_session_dict = {"MRI_v1": "ses-01", "MRI_v2": "ses-02", "MRI_v3": "ses-03"}

manifest_cols = ["visit","session","datatype"]
avail_datatypes = "['anat','dwi','fmap','func']"

current_manifest_df = pd.DataFrame()

for visit_label in visit_labels:
    visit_participant_ids = visit_participant_dict[visit_label]
    print(f"visit_id: {visit_label}, n_participants: {len(visit_participant_ids)}")

    _df = pd.DataFrame(index=visit_participant_ids, columns=manifest_cols)

    session = visit_session_dict[visit_label]

    _df.loc[visit_participant_ids,"visit"] = visit_label
    _df.loc[visit_participant_ids,"session"] = session
    _df.loc[visit_participant_ids,"datatype"] = avail_datatypes
    
    current_manifest_df = pd.concat([current_manifest_df, _df], axis=0)

current_manifest_df

visit_id: MRI_v1, n_participants: 303
visit_id: MRI_v2, n_participants: 52
visit_id: MRI_v3, n_participants: 4


,visit,session,datatype
PD00016,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00020,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00032,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00048,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
PD00119,MRI_v1,ses-01,"['anat','dwi','fmap','func']"
...,...,...,...
MNI0359,MRI_v2,ses-02,"['anat','dwi','fmap','func']"
PD00849,MRI_v3,ses-03,"['anat','dwi','fmap','func']"
PD00955,MRI_v3,ses-03,"['anat','dwi','fmap','func']"
PD01199,MRI_v3,ses-03,"['anat','dwi','fmap','func']"


### Save update CSV

In [25]:
save_current_manifest = True
if save_current_manifest:
    print(f"Saving new nipoppy manifest here: {current_manifest_csv}")
    current_manifest_df = current_manifest_df.reset_index().rename(columns={"index":"participant_id"})
    current_manifest_df.to_csv(current_manifest_csv,index=None)

Saving new nipoppy manifest here: /home/nikhil/projects/Parkinsons/qpn/releases/March_2024/tabular/manifest.csv
